In [28]:
'''
DEFINE MASTER FUNCTIONS 
'''

# Census 1 
def define_geography(map_name, gdb, focus_points, focus_point_name, selection_distance, shp_folder): 

    # Clear map 
    mp        = define_map(map_name)
    mp        = clear_map(mp)

    # Export block group selections 
    fc_path   = select_block_groups_by_focus_point(focus_point_name, focus_points, block_groups, selection_distance, mp, overwrite = True, add_to_map = True)
    
    shp_path  = export_shapefiles(fc_path, shp_folder)
    
    return shp_path

# Census 2
def view_acs_codes(json_url, filter_words, blocked_words): 
    
    # Read JSON 
    json_dictionary         = read_census_json(json_url)
    group_dictionary        = return_group_dictionary(json_dictionary)

    # View the contents 
    acs_dataframe          = print_group_dictionary(group_dictionary, filter_words, blocked_words, print_groups = False, print_codes = False)
    
    return json_dictionary, group_dictionary, acs_dataframe 

def return_acs_dictionary(input_group_code, json_dictionary, group_dictionary):
    
    # Find relevant codes 
    variable_codes        = find_variable_codes(input_group_code, group_dictionary)
    code_label_dictionary = create_code_label_dictionary(variable_codes, json_dictionary)
    
    return variable_codes, code_label_dictionary

# Census 3 
def retrieve_acs_json(variable_codes, shp_path, api_key): 
    
    variable_string = create_variable_string(variable_codes)
    api_urls        = construct_api_urls(shp_path, api_key, variable_string)
    json_list       = call_census_api(api_urls)
    
    return json_list

def json_to_dataframe(json_list): 
    
    census_dataframe  = json_list_to_dataframe(json_list)
    census_dataframe  = change_columns_to_integers(census_dataframe)
    census_dataframe  = calculate_percentage_columns(census_dataframe)
    
    return census_dataframe 

def create_geodataframe(shp_path, keep_columns): 
    
    # Create geodataframe 
    geodataframe = gpd.read_file(shp_path)
    
    # Drop unnecessary columns 
    for column in geodataframe.columns: 

        if column not in keep_columns: 
            try: 
                geodataframe = geodataframe.drop(columns = column)
            except: 
                pass
            
    return geodataframe

def merge_dataframe_with_geometry(geodataframe, dataframe, output_shp_path): 
    
    merge_geodataframe = geodataframe.merge(census_dataframe, on = 'GEOID', how = 'left')
    merge_geodataframe.to_file(output_shp_path + ".shp", driver = 'ESRI Shapefile')
    
    print(output_shp_path)
    
    return merge_geodataframe, output_shp_path

In [29]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

In [30]:
'''
PARAMETERS
'''

# Imports 
import os 
import pandas as pd
import geopandas as gpd 
import requests 
from pandas import option_context

# Map 
map_name   = 'Census'
add_to_map = True 

# Input Data  
block_groups = r"https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Tracts_Blocks/MapServer/1"
focus_points = r"C:\DALE\Personal\Minnesota\MyProject\MyProject.gdb\Focus_Points"

# Output Data 
gdb          = r"C:\DALE\Personal\Minnesota\MyProject\MyProject.gdb"
shp_folder   = r"C:\DALE\Personal\Minnesota\MyProject\Shapefiles"
output_shp   = os.path.join(shp_folder, 'Minneapolis_MonthlyHousingCosts')

# Selection criteria 
focus_point_name   = 'SMU Minneapolis'
selection_distance = '10 Miles'

# Variables searching 
filter_words       = []
blocked_words      = []

# API call 
api_key            = r"1749dc1d87964116107a80cd7d76fca300dda59f"
base_url           = r"https://api.census.gov/data/2022/acs/acs5"
json_url           = fr"{base_url}/variables.json"

input_group_code   = 'B08303'

# Geodataframe 
keep_columns       = ['GEOID', 'geometry']

In [31]:
'''
RUN MASTER FUNCTIONS (VIEW CENSUS CONCEPTS)
'''
filter_words       = ['Presence of Own Children']
blocked_words      = ['(']

json_dictionary, group_dictionary, acs_dataframe = view_acs_codes(json_url, filter_words, blocked_words)

with option_context('display.max_colwidth', None): 
    print(acs_dataframe)

JSON dictionary returned
Group dictionary returned
Group dictionary filtered and passed to dataframe
  Group Codes                                                                                                                       Concepts
0      B23003  Presence of Own Children Under 18 Years by Age of Own Children Under 18 Years by Employment Status for Females 20 to 64 Years
1      B11013                                                                      Subfamily Type by Presence of Own Children Under 18 Years
2      B23009                    Presence of Own Children Under 18 Years by Family Type by Number of Workers in Family in the Past 12 Months
3      B25012                                                                                Tenure by Families and Presence of Own Children
4      B23010                Presence of Own Children Under 18 Years in Married-Couple Families by Work Experience of Householder and Spouse
5      B23007                                        

In [32]:
'''
RUN MASTER FUNCTIONS (CREATE OUTPUT DATA)
'''
shp_path                                         = define_geography(map_name, gdb, focus_points, focus_point_name, selection_distance, shp_folder)
json_dictionary, group_dictionary, acs_dataframe = view_acs_codes(json_url, filter_words, blocked_words)
variable_codes, code_label_dictionary            = return_acs_dictionary(input_group_code, json_dictionary, group_dictionary)
json_list                                        = retrieve_acs_json(variable_codes, shp_path, api_key)

SMU Minneapolis
 >>> 813
﻿
C:\DALE\Personal\Minnesota\MyProject\Shapefiles\SMU_Minneapolis_Block_Groups.shp
JSON dictionary returned
Group dictionary returned
Group dictionary filtered and passed to dataframe
Call #1 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1.44
 
Call #2 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 2.72
 
Call #3 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 4.26
 
Call #4 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 5.74
 
Call #5 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 7.09
 
Call #6 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 8.32
 
Call #7 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 9.71
 
Call #8 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 11.02
 
Call #9 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 12.43
 
Call #10 / 813
>>> Sent request
Checking status:
>>

Checking status:
>>> Good
Elapsed time:
>>> 244.07
 
Call #192 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 245.33
 
Call #193 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 246.64
 
Call #194 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 247.88
 
Call #195 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 249.14
 
Call #196 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 250.4
 
Call #197 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 251.76
 
Call #198 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 253.03
 
Call #199 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 254.29
 
Call #200 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 255.57
 
Call #201 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 256.98
 
Call #202 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 258.31
 
Ca

Call #382 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 495.43
 
Call #383 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 496.65
 
Call #384 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 497.87
 
Call #385 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 499.09
 
Call #386 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 500.35
 
Call #387 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 501.59
 
Call #388 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 502.83
 
Call #389 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 504.05
 
Call #390 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 505.39
 
Call #391 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 506.62
 
Call #392 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 507.85
 
Call #393 / 813
>>> Sent request
Checking status:
>>> 

Elapsed time:
>>> 754.32
 
Call #573 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 755.57
 
Call #574 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 756.85
 
Call #575 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 758.12
 
Call #576 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 759.38
 
Call #577 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 760.65
 
Call #578 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 761.91
 
Call #579 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 763.35
 
Call #580 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 764.72
 
Call #581 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 765.98
 
Call #582 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 767.35
 
Call #583 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 768.68
 
Call #584 / 813
>>> Sent re

>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 999.65
 
Call #764 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1000.92
 
Call #765 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1002.15
 
Call #766 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1003.45
 
Call #767 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1004.71
 
Call #768 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1005.96
 
Call #769 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1007.2
 
Call #770 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1008.42
 
Call #771 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1009.66
 
Call #772 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1010.9
 
Call #773 / 813
>>> Sent request
Checking status:
>>> Good
Elapsed time:
>>> 1012.18
 
Call #774 / 813
>>> Sent request
Checking status:
>>> Good
Ela

In [33]:
geodataframe                                     = create_geodataframe(shp_path, keep_columns)
merge_geodataframe, output_shp_path              = merge_dataframe_with_geometry(geodataframe, census_dataframe, output_shp)

[28]:75: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


C:\DALE\Personal\Minnesota\MyProject\Shapefiles\Minneapolis_MonthlyHousingCosts


In [34]:
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

In [35]:
'''
1. Trim unnecessary columns from geodataframe 
2. Format output columns so that they have clear names 
3. Create function to group columns if needed 
'''

'\n1. Trim unnecessary columns from geodataframe \n2. Format output columns so that they have clear names \n3. Create function to group columns if needed \n'